<a href="https://colab.research.google.com/github/omkarwazulkar/GoogleColab/blob/main/Titanic_NN_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load dataset (Titanic as an example)
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(url)

In [29]:
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
df['Sex'] = df['Sex'].map({'male':0, 'female':1})

In [31]:
# Fill missing Age with median
df['Age'] = df['Age'].fillna(df['Age'].median())

# Fill missing Embarked with mode
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])

# Drop Cabin, Name, Ticket, PassengerId
df = df.drop(columns=['Cabin', 'Name', 'Ticket', 'PassengerId'])

In [33]:
df = pd.get_dummies(df, columns=['Embarked'], drop_first=True)

In [34]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S
0,0,3,0,22.0,1,0,7.2500,False,True
1,1,1,1,38.0,1,0,71.2833,False,False
2,1,3,1,26.0,0,0,7.9250,False,True
3,1,1,1,35.0,1,0,53.1000,False,True
4,0,3,0,35.0,0,0,8.0500,False,True


In [35]:
X = df.drop('Survived', axis=1).values
Y = df['Survived'].values

In [37]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X)

In [38]:
from sklearn.model_selection import train_test_split
import torch

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
Y_train = torch.tensor(Y_train, dtype=torch.float32).view(-1,1)
Y_test = torch.tensor(Y_test, dtype=torch.float32).view(-1,1)

In [41]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [42]:
import torch.nn as nn

class TitanicNN(nn.Module):
    def __init__(self, input_dim):
        super(TitanicNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 32)
        self.bn1 = nn.BatchNorm1d(32)
        self.fc2 = nn.Linear(32, 16)
        self.bn2 = nn.BatchNorm1d(16)
        self.fc3 = nn.Linear(16, 8)
        self.fc4 = nn.Linear(8, 1)

        self.relu = nn.LeakyReLU(0.1)
        self.dropout = nn.Dropout(0.3)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.fc3(x)
        x = self.relu(x)
        x = self.fc4(x)
        x = self.sigmoid(x)
        return x

In [43]:
input_dim = X_train.shape[1]
model = TitanicNN(input_dim).to(device)

criterion = nn.BCELoss()  # Binary cross entropy
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Move data to device
X_train, Y_train = X_train.to(device), Y_train.to(device)
X_test, Y_test = X_test.to(device), Y_test.to(device)

In [47]:
model.eval()
with torch.no_grad():
    predictions = model(X_test)
    predicted_classes = (predictions > 0.5).float()

    print("Showing first 5 test cases predictions with actual labels:\n")
    for i in range(5):  # show first 5 predictions
        input_features = X_test[i].cpu().numpy()
        prob = predictions[i].item()
        pred_class = predicted_classes[i].item()
        actual = Y_test[i].cpu().item()
        print(f"Input: {input_features}")
        print(f"Predicted Probability: {prob:.4f}")
        print(f"Predicted Class: {pred_class}")
        print(f"Actual Class: {actual}\n")

Showing first 5 test cases predictions with actual labels:

Input: [ 0.82737726 -0.73769516 -0.1046374   0.43279338  0.76762986 -0.34145224
 -0.30756235 -1.6238025 ]
Predicted Probability: 0.0567
Predicted Class: 0.0
Actual Class: 1.0

Input: [-0.36936483 -0.73769516  0.12591213 -0.4745452  -0.4736736  -0.43700743
 -0.30756235  0.6158384 ]
Predicted Probability: 0.1297
Predicted Class: 0.0
Actual Class: 0.0

Input: [ 0.82737726 -0.73769516 -0.71943617 -0.4745452  -0.4736736  -0.48885426
 -0.30756235  0.6158384 ]
Predicted Probability: 0.0991
Predicted Class: 0.0
Actual Class: 0.0

Input: [-0.36936483  1.3555735  -1.795334   -0.4745452   0.76762986  0.01602302
 -0.30756235  0.6158384 ]
Predicted Probability: 0.9778
Predicted Class: 1.0
Actual Class: 1.0

Input: [ 0.82737726  1.3555735  -1.1805352   0.43279338 -0.4736736  -0.42207354
 -0.30756235 -1.6238025 ]
Predicted Probability: 0.5598
Predicted Class: 1.0
Actual Class: 1.0



In [49]:
from torch.utils.data import TensorDataset, DataLoader

batch_size = 32

# Create datasets
train_dataset = TensorDataset(X_train, Y_train)
test_dataset = TensorDataset(X_test, Y_test)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [50]:
class TitanicNN(nn.Module):
    def __init__(self, input_dim):
        super(TitanicNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 32)
        self.bn1 = nn.BatchNorm1d(32)
        self.fc2 = nn.Linear(32, 16)
        self.bn2 = nn.BatchNorm1d(16)
        self.fc3 = nn.Linear(16, 8)
        self.fc4 = nn.Linear(8, 1)

        self.relu = nn.LeakyReLU(0.1)
        self.dropout = nn.Dropout(0.3)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.fc3(x)
        x = self.relu(x)
        x = self.fc4(x)
        x = self.sigmoid(x)
        return x


In [51]:
import torch
from sklearn.metrics import precision_score, recall_score, f1_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TitanicNN(X_train.shape[1]).to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


In [52]:
import numpy as np

epochs = 1000
patience = 20
best_val_loss = np.inf
counter = 0

for epoch in range(epochs):
    model.train()
    train_loss = 0
    for batch_X, batch_Y in train_loader:
        batch_X, batch_Y = batch_X.to(device), batch_Y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_Y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * batch_X.size(0)

    train_loss /= len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch_X, batch_Y in test_loader:
            batch_X, batch_Y = batch_X.to(device), batch_Y.to(device)
            outputs = model(batch_X)
            loss = criterion(outputs, batch_Y)
            val_loss += loss.item() * batch_X.size(0)

            preds = (outputs > 0.5).float()
            all_preds.append(preds.cpu())
            all_labels.append(batch_Y.cpu())

    val_loss /= len(test_loader.dataset)
    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)

    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)

    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, "
          f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

    # Early Stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
        torch.save(model.state_dict(), "best_titanic_model.pth")
    else:
        counter += 1
        if counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break


Epoch 1, Train Loss: 0.5945, Val Loss: 0.4669, Precision: 0.7606, Recall: 0.7297, F1: 0.7448
Epoch 2, Train Loss: 0.4905, Val Loss: 0.4542, Precision: 0.8095, Recall: 0.6892, F1: 0.7445
Epoch 3, Train Loss: 0.4577, Val Loss: 0.4308, Precision: 0.7941, Recall: 0.7297, F1: 0.7606
Epoch 4, Train Loss: 0.4760, Val Loss: 0.4289, Precision: 0.8182, Recall: 0.7297, F1: 0.7714
Epoch 5, Train Loss: 0.4521, Val Loss: 0.4307, Precision: 0.8060, Recall: 0.7297, F1: 0.7660
Epoch 6, Train Loss: 0.4599, Val Loss: 0.4325, Precision: 0.8421, Recall: 0.6486, F1: 0.7328
Epoch 7, Train Loss: 0.4496, Val Loss: 0.4391, Precision: 0.8030, Recall: 0.7162, F1: 0.7571
Epoch 8, Train Loss: 0.4427, Val Loss: 0.4115, Precision: 0.8548, Recall: 0.7162, F1: 0.7794
Epoch 9, Train Loss: 0.4456, Val Loss: 0.4218, Precision: 0.8548, Recall: 0.7162, F1: 0.7794
Epoch 10, Train Loss: 0.4484, Val Loss: 0.4241, Precision: 0.8500, Recall: 0.6892, F1: 0.7612
Epoch 11, Train Loss: 0.4363, Val Loss: 0.4352, Precision: 0.8500, Re

In [54]:
# Load the best model
model.load_state_dict(torch.load("best_titanic_model.pth"))
model.eval()

all_preds = []
with torch.no_grad():
    for batch_X, _ in test_loader:
        batch_X = batch_X.to(device)
        outputs = model(batch_X)
        preds = (outputs > 0.5).float()
        all_preds.append(preds.cpu())
all_preds = torch.cat(all_preds)

# Compare with actual labels
for i in range(5):
    print(f"Pred: {all_preds[i].item()}, Actual: {Y_test[i].item()}")

Pred: 0.0, Actual: 1.0
Pred: 0.0, Actual: 0.0
Pred: 0.0, Actual: 0.0
Pred: 1.0, Actual: 1.0
Pred: 1.0, Actual: 1.0
